In [1]:
%pip install -q langchain langchain-groq python-dotenv faiss-cpu langchain_huggingface beautifulsoup4 langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
#Using google.colab secrets
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    print("🛑 Groq API Key not found. Please make sure to set it up.")
else:
    print("✅ Groq API Key configured.")

✅ Groq API Key configured.


In [3]:
# Initialize the Chat model with Groq
# We'll use Llama3 8B, a fast and competent model
from langchain_groq import ChatGroq


llm = ChatGroq(model_name="llama-3.1-8b-instant", groq_api_key=api_key)


print("Language Model initialized with Groq.")

Language Model initialized with Groq.


Let´s decide who is worth it in the market, to buy or not to buy

In [18]:
#Firstly, let´s load and clean the market data
import json
market_file_path = "market/market_20250930.json"
with open(market_file_path, "r", encoding="utf-8") as f:
    market_data = json.load(f)

# Extract only the fields you need
extracted = []
for item in market_data:
    extracted.append({
        "player_id": item["playerMaster"]["id"],
        "salePrice": item.get("salePrice"),
        "numberOfBids": item.get("numberOfBids", 0)  # default to 0 if missing
    })

In [26]:
#Firstly, let´s load and clean the market data
import json
import requests

from download_pipeline import get_autorization_headers, make_request

header = get_autorization_headers()
market_file_path = "market/market_20250930.json"
with open(market_file_path, "r", encoding="utf-8") as f:
    market_data = json.load(f)

# Extract only the fields you need
extracted = []
for item in market_data:
    player_id = item["playerMaster"]["id"]
    entry = {
        "player_id": player_id,
        "salePrice": item.get("salePrice"),
        "numberOfBids": item.get("numberOfBids", 0),
    }

    # Build the API URL
    url = f"https://api-fantasy.llt-services.com/api/v3/player/{player_id}/market-value?x-lang=en"
    try:
        data = make_request("GET", url, headers=header)
        entry["marketValueInfo"] = data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for player {player_id}: {e}")
        entry["marketValueInfo"] = None

    extracted.append(entry)

2025-09-30 23:13:25,011 - INFO - Loading environment variables...
2025-09-30 23:13:25,014 - INFO - Refreshing the token...
2025-09-30 23:13:25,397 - INFO - Token refreshed successfully.


In [30]:
len(extracted[0]["marketValueInfo"])

51